# Image segmentation

## Data

source: https://www.kaggle.com/datasets/kmader/finding-lungs-in-ct-data

In [ ]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
import glob
from skimage import io
from skimage.transform import resize
from sklearn.model_selection import train_test_split

### Load

In [2]:
# load images
img_2d = sorted(glob.glob("data/2d_images" + "/*.tif"))
mask_2d = sorted(glob.glob("data/2d_masks" + "/*.tif"))
img_3d = sorted(glob.glob("data/3d_images/imgs" + "/*.nii"))
mask_3d = sorted(glob.glob("data/3d_images/masks" + "/*.nii"))

In [3]:
print(f"There are {len(img_2d)} 2d images and {len(mask_2d)} 2d masks.")

There are 267 2d images and 267 2d masks.


In [4]:
print(f"There are {len(img_3d)} 3d images and {len(mask_3d)} 3d masks.")

There are 4 3d images and 4 3d masks.


As there is a low number of 3d images and masks I will only work with 2d images.

### Process

In [50]:
# # show the image
# fig, ax = plt.subplots(1, 2)
# ax[0].imshow(x_data[0].squeeze(), cmap='gray')
# ax[1].imshow(y_data[0].squeeze(), cmap='gray')

## Model

I will explore different layers' structures and analyse their results.

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.callbacks import ReduceLROnPlateau

### IMG_SIZE 128 -> 5 conv layers

In [44]:
IMG_SIZE = 128

In [53]:
def convert_to_array(path, size):
    # load
    data = io.imread(path)
    # resize to 1 channel
    data = resize(data, output_shape=(size, size, 1), preserve_range=True)
    # return the image to save it
    return data

def normalize_imgs(img_2d, mask_2d, size):
    ## create the np.array where we will save the data
    x_data, y_data = np.empty((2, len(img_2d), size, size, 1), dtype=np.float32)
    ## save the processed imgs and their masks
    for i, img_path in enumerate(img_2d):
        x_data[i] = convert_to_array(img_path, size)
    for i, mask_path in enumerate(mask_2d):
        y_data[i] = convert_to_array(mask_path, size)
    ## scale
    x_data /= 255
    y_data /= 255
    return x_data, y_data

In [ ]:
x_data, y_data = normalize_imgs(img_2d, mask_2d, IMG_SIZE)
# first, we split data in train and test
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

In [47]:
# Define input layer
inputs = Input(shape=(128, 128, 1))

# Encoder: 4 blocks conv + pool augmenting the filters
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

# Bottleneck layer
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)

# Decoder: 4 blocks upsampling + conv reducing the filters
up4 = UpSampling2D(size=(2, 2))(conv3)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up4)

up5 = UpSampling2D(size=(2, 2))(conv4)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up5)

# Output layer
outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv5)

# Build model
model_k3 = Model(inputs=inputs, outputs=outputs)

# Compile model
model_k3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model_k3.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_57 (Conv2D)              │ (None, 128, 128, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_58 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_17 (UpSampling2D) │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_60 (Conv2D)              │ (None, 64, 64, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_18 (UpSampling2D) │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_61 (Conv2D)              │ (None, 128, 128, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_62 (Conv2D)              │ (None, 128, 128, 1)    │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 184,961 (722.50 KB)

 Trainable params: 184,961 (722.50 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=1, min_lr=1e-05)
# train model
history = model_k3.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=32, callbacks=[reduce_lr])

Epoch 1/2
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 683ms/step - accuracy: 0.7088 - loss: 0.5702 - val_accuracy: 0.7119 - val_loss: 0.4552 - learning_rate: 0.0010
Epoch 2/2
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 616ms/step - accuracy: 0.7105 - loss: 0.4278 - val_accuracy: 0.7119 - val_loss: 0.3470 - learning_rate: 0.0010


### IMG_SIZE 256 -> 7 conv layers

In [51]:
IMG_SIZE = 256

In [54]:
x_data, y_data = normalize_imgs(img_2d, mask_2d, IMG_SIZE)
# first, we split data in train and test
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

In [55]:
# Define input layer
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 1))

# Encoder: three blocks conv + pool augmenting the filters
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

# Bottleneck layer
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)

# Decoder: 3 blocks upsampling + conv reducing the filters
up5 = UpSampling2D(size=(2, 2))(conv4)
conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(up5)

up6 = UpSampling2D(size=(2, 2))(conv5)
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(up6)

up7 = UpSampling2D(size=(2, 2))(conv6)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up7)

# Output layer
outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv7)

# Build model
model_k1 = Model(inputs=inputs, outputs=outputs)

# Compile model
model_k1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model_k1.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_63 (Conv2D)              │ (None, 256, 256, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_64 (Conv2D)              │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_65 (Conv2D)              │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_66 (Conv2D)              │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_19 (UpSampling2D) │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_67 (Conv2D)              │ (None, 64, 64, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_20 (UpSampling2D) │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_68 (Conv2D)              │ (None, 128, 128, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_21 (UpSampling2D) │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_69 (Conv2D)              │ (None, 256, 256, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_70 (Conv2D)              │ (None, 256, 256, 1)    │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 775,169 (2.96 MB)

 Trainable params: 775,169 (2.96 MB)

 Non-trainable params: 0 (0.00 B)

In [56]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=1, min_lr=1e-05)
# train model
history = model_k1.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=32, callbacks=[reduce_lr])

Epoch 1/2
5/7 ━━━━━━━━━━━━━━━━━━━━ 7s 4s/step - accuracy: 0.7071 - loss: 0.6321 

KeyboardInterrupt: 

#### 512 IMG_Size -> 9 conv layers

In [25]:
IMG_SIZE = 512

In [ ]:
x_data, y_data = normalize_imgs(img_2d, mask_2d, IMG_SIZE)
# first, we split data in train and test
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

In [33]:
# Define input layer
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 1))

# Encoder: 4 blocks conv + pool augmenting the filters
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

# Bottleneck layer
conv5 = Conv2D(IMG_SIZE, (3, 3), activation='relu', padding='same')(pool4)

# Decoder: 4 blocks upsampling + conv reducing the filters
up6 = UpSampling2D(size=(2, 2))(conv5)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)

up7 = UpSampling2D(size=(2, 2))(conv6)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)

up8 = UpSampling2D(size=(2, 2))(conv7)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)

up9 = UpSampling2D(size=(2, 2))(conv8)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)

# Output layer
outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

# Build model
model_k2 = Model(inputs=inputs, outputs=outputs)

# Compile model
model_k2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model_k1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 64, 64, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 256, 256, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 256, 256, 1)    │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,325,509 (8.87 MB)

 Trainable params: 775,169 (2.96 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,550,340 (5.91 MB)

In [34]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=1, min_lr=1e-05)
# train model
history = model_k2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=32, callbacks=[reduce_lr])

Epoch 1/2
7/7 ━━━━━━━━━━━━━━━━━━━━ 211s 28s/step - accuracy: 0.6389 - loss: 0.6731 - val_accuracy: 0.7650 - val_loss: 0.5038 - learning_rate: 0.0010
Epoch 2/2
7/7 ━━━━━━━━━━━━━━━━━━━━ 210s 29s/step - accuracy: 0.7680 - loss: 0.4928 - val_accuracy: 0.7650 - val_loss: 0.4476 - learning_rate: 0.0010
